In [0]:
#import required libraries 

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [0]:
import os

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#go to working directory
cd "/content/drive/My Drive/ColabNotebooks/PicToRecipe"

/content/drive/My Drive/ColabNotebooks/PicToRecipe


In [0]:
#define the veriables
img_width, img_height = 224, 224
train_data_dir = "DataSet/Training_set"
validation_data_dir = "DataSet/Test_set"

In [0]:
path, data_dir_list, files = next(os.walk(train_data_dir))
data_dir_list

['Red Tomato',
 'Broccoli',
 'Spring Onion',
 'Mushroom',
 'Potato',
 'Carrot',
 'Corn',
 'Spinach',
 'Eggplant',
 'Onion',
 'Coliflower',
 'Green Tomato',
 'Brussels Sprouts',
 'Green Beans',
 'Avocado',
 'Cabbage',
 'cucumber',
 'beet',
 'sweetpotato',
 'Okra',
 'peas',
 'Pumpkin']

In [0]:
#Calculate Train samples
train_samples = 0

path, dirs, files = next(os.walk(train_data_dir))
nb_classsifires = len(dirs)
print(nb_classsifires)
for d in dirs:
    path, nub_classifires, files = next(os.walk(train_data_dir + "/" + d))
    train_samples = train_samples+len(files)
    
print(train_samples ) 

22
1030


In [0]:
test_samples = 0

path, dirs, files = next(os.walk(validation_data_dir))
for d in dirs:
    path, nub_classifires, files = next(os.walk(validation_data_dir + "/" + d))
    test_samples = test_samples+len(files)
    
print(test_samples) 

276


In [0]:
batch_size = 10
epochs = 50

In [0]:
#VGG19 model, with weights pre-trained on ImageNet.
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#model = applications.VGG19(weights = "imagenet", include_top=True, classes=1000, input_shape = None)

80142336/80134624 [==============================] - 1s 0us/step


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

In [0]:
#from keras.preprocessing.image import img_to_array

In [0]:
#x = model.get_layer('fc2').output
#x = Dense(128, activation="relu")(x)
#x = Dropout(0.5)(x)
#x = Dense(128, activation="relu")(x)
#x = Dropout(0.2)(x)
#x = Dense(128, activation="relu")(x)
#predictions = Dense(nb_classsifires, activation="softmax")(x)

In [0]:
x = model.output
x = Flatten()(x) 
x = Dense(128, activation="relu")(x) 
x = Dropout(0.5)(x) 
x = Dense(128, activation="relu")(x) 
x = Dropout(0.2)(x) 
x = Dense(128, activation="relu")(x) 
predictions = Dense(nb_classsifires, activation="softmax")(x)

In [0]:
# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

In [0]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [0]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   fill_mode = "nearest",
                                   zoom_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

In [0]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True,
                                  fill_mode = "nearest",
                                  zoom_range = 0.3,
                                  width_shift_range = 0.3,
                                  height_shift_range=0.3,
                                  rotation_range=30)

In [0]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, img_width),
                                                    batch_size = batch_size, 
                                                    class_mode = "categorical")

Found 1027 images belonging to 22 classes.


In [0]:
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size = (img_height, img_width),
                                                        class_mode = "categorical")

Found 275 images belonging to 22 classes.


In [0]:
train_generator.class_indices

{'Avocado': 0,
 'Broccoli': 1,
 'Brussels Sprouts': 2,
 'Cabbage': 3,
 'Carrot': 4,
 'Coliflower': 5,
 'Corn': 6,
 'Eggplant': 7,
 'Green Beans': 8,
 'Green Tomato': 9,
 'Mushroom': 10,
 'Okra': 11,
 'Onion': 12,
 'Potato': 13,
 'Pumpkin': 14,
 'Red Tomato': 15,
 'Spinach': 16,
 'Spring Onion': 17,
 'beet': 18,
 'cucumber': 19,
 'peas': 20,
 'sweetpotato': 21}

In [0]:
import pandas as pd
d = train_generator.class_indices

In [0]:
d.items()

dict_items([('Avocado', 0), ('Broccoli', 1), ('Brussels Sprouts', 2), ('Cabbage', 3), ('Carrot', 4), ('Coliflower', 5), ('Corn', 6), ('Eggplant', 7), ('Green Beans', 8), ('Green Tomato', 9), ('Mushroom', 10), ('Okra', 11), ('Onion', 12), ('Potato', 13), ('Pumpkin', 14), ('Red Tomato', 15), ('Spinach', 16), ('Spring Onion', 17), ('beet', 18), ('cucumber', 19), ('peas', 20), ('sweetpotato', 21)])

In [0]:
labels = pd.DataFrame(list(d.items()), columns=['Label', 'Index'])

In [0]:
labels

,Label,Index
0,Avocado,0
1,Broccoli,1
2,Brussels Sprouts,2
3,Cabbage,3
4,Carrot,4
5,Coliflower,5
6,Corn,6
7,Eggplant,7
8,Green Beans,8
9,Green Tomato,9


In [0]:
labels.to_csv("labels.csv", index=False)

In [0]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg19_pictorecipe_model.h5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [0]:
# Train the model 
model_final.fit_generator(train_generator,
                          steps_per_epoch = train_samples/4,
                          epochs = epochs,
                          validation_data = validation_generator,
                          validation_steps = test_samples/2,
                          callbacks = [checkpoint, early])

Epoch 1/50
258/257 [==============================] - 416s 2s/step - loss: 3.0966 - acc: 0.0562 - val_loss: 3.0845 - val_acc: 0.0623
Epoch 2/50
258/257 [==============================] - 210s 814ms/step - loss: 3.0553 - acc: 0.0767 - val_loss: 3.0668 - val_acc: 0.0557
Epoch 3/50
258/257 [==============================] - 210s 815ms/step - loss: 2.9732 - acc: 0.1068 - val_loss: 2.9427 - val_acc: 0.1024
Epoch 4/50
258/257 [==============================] - 211s 818ms/step - loss: 2.7857 - acc: 0.1513 - val_loss: 2.7587 - val_acc: 0.1668
Epoch 5/50
258/257 [==============================] - 211s 816ms/step - loss: 2.5842 - acc: 0.1915 - val_loss: 2.3791 - val_acc: 0.2781
Epoch 6/50
258/257 [==============================] - 209s 810ms/step - loss: 2.3208 - acc: 0.2746 - val_loss: 1.9879 - val_acc: 0.4197
Epoch 7/50
258/257 [==============================] - 210s 815ms/step - loss: 2.0691 - acc: 0.3557 - val_loss: 1.6895 - val_acc: 0.5229
Epoch 8/50
258/257 [==============================]